<a href="https://colab.research.google.com/github/iIsunnyIi/PractisePro/blob/main/stacking_model_for_facies_classification_goldy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Required Imports
import pandas as pd
#from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
#from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score
import numpy as np
import pickle 

In [2]:
# Reading data
data=pd.read_csv('training_data.csv')

data.drop(columns=['Formation','Well Name'], inplace=True) # Dropping categorical data

# Dividing Feature and Target columns
x=data.drop(columns=['Facies'])
y=data['Facies']

FileNotFoundError: ignored

In [ ]:
# Creating model objects
#xgb=XGBClassifier()
rfr=RandomForestClassifier()
svm=SVC()
lgbm=LGBMClassifier()

In [ ]:
# Using Stacking Object
def stacking(x,y,meta_alg,algo_list=list):
    
    ''' x : Feature Columns
        y : Target Column
        meta_alg : Algorithm which will be used to blend rest of algorithm
        algo_list: Base algorithms used in stacking '''
    
    #spliting data into training set and holding 50 % of data
    train,val_train,test,val_test=train_test_split(x,y,test_size=0.5)
    
    # spliting training data into training and test data
    x_train,x_test,y_train,y_test=train_test_split(train,test,test_size=0.2)
    
    # Here we are creating input data for Meta-Algo
    predict=[]
    # Fitting base models with train data
    for i,algo in enumerate(algo_list):
        z=algo.fit(x_train,y_train)
        filename='base_model_'+str(i)+'.sav' # Saving Base models
        with open(filename,'wb') as f:
            pickle.dump(z,f)
        # Here we are giving predictions on base models for validation data
        i=z.predict(val_train)
        predict.append(i)
        predict_val=np.column_stack(predict) # Blending all predictions to feed meta-algo
    
    # Here we are creating input to check accuracy on Trained Meta-Algo
    test=[]
    for i,algo in enumerate(algo_list):
        i=algo.predict(x_test)
        test.append(i)
        predict_test=np.column_stack(test)
        
    # Fitting meta algorithm with 
    model=meta_alg.fit(predict_val,val_test)
    
    # Uncheck below 3 lines to apply Cross-validation
    #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    #scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    #print(score)
    print('Test Score of meta algorithm is', meta_alg.score(predict_test,y_test))
    print('Train Score of meta algorithm is', meta_alg.score(predict_val,val_test))
    
    #Saving Model for 
    filename='stacking_model.sav'
    with open( filename, 'wb') as f:
        pickle.dump(model,f)
    
    return model

In [ ]:
stacking(x,y,rfr,[rfr,svm,lgbm])

Test Score of meta algorithm is 0.7623456790123457
Train Score of meta algorithm is 0.7784653465346535


RandomForestClassifier()

In [ ]:
def predict_single(x):
    with open('base_model_0.sav','rb') as f:
        base_model_0=pickle.load(f)
    with open('base_model_1.sav','rb') as f:
        base_model_1=pickle.load(f)
    with open('base_model_2.sav','rb') as f:
        base_model_2=pickle.load(f)
    with open('stacking_model.sav','rb') as f:
        stacked_model=pickle.load(f)
    
    base_models=[base_model_0,base_model_1,base_model_2]
    
    x=np.array(x).reshape(1,-1)
    
    test=[]
    for i,model in enumerate(base_models):
        i=model.predict(x)
        test.append(i)
        prediction=np.column_stack(test)
        
    prediction=stacked_model.predict(prediction)
    result='Facies type is: {}'.format(prediction[0])
    return result
        

In [ ]:
data.iloc[879,0]

6

In [ ]:
x=data.iloc[879,1:]

In [ ]:
predict_single(x)

'Facies type is: 6'

In [ ]:
def predict_multiple(x):
    with open('base_model_0.sav','rb') as f:
        base_model_0=pickle.load(f)
    with open('base_model_1.sav','rb') as f:
        base_model_1=pickle.load(f)
    with open('base_model_2.sav','rb') as f:
        base_model_2=pickle.load(f)
    with open('stacking_model.sav','rb') as f:
        stacked_model=pickle.load(f)
    
    base_models=[base_model_0,base_model_1,base_model_2]
    
    #file=Path_of_file_folder+filename
    #data=pd.read(file)
    
    test=[]
    for i,model in enumerate(base_models):
        i=model.predict(x)
        test.append(i)
        prediction=np.column_stack(test)
        
    prediction=stacked_model.predict(prediction)
    
    return prediction

In [ ]:
x=data.iloc[56:89,1:]
x

,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
56,2821.0,43.67,0.876,3.6,8.225,4.1,2,0.741
57,2821.5,55.23,0.826,5.1,7.875,4.3,2,0.722
58,2822.0,61.24,0.777,9.7,8.575,4.3,2,0.704
59,2822.5,54.46,0.728,12.8,10.660,4.4,2,0.685
60,2823.0,43.23,0.699,14.9,12.375,4.0,2,0.667
61,2823.5,44.59,0.682,16.0,13.540,3.9,2,0.648
62,2824.0,47.48,0.676,16.5,12.890,3.7,2,0.630
63,2824.5,61.10,0.671,15.2,11.785,3.7,2,0.611
64,2825.0,64.82,0.680,9.1,8.970,4.5,2,0.574
65,2825.5,55.77,0.710,6.1,8.930,5.1,2,0.556


In [ ]:
predict_multiple(x)

array([6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], dtype=int64)

In [ ]:
data.iloc[56:89,0]

56    6
57    6
58    6
59    6
60    7
61    7
62    4
63    4
64    4
65    4
66    4
67    6
68    6
69    6
70    6
71    6
72    6
73    6
74    6
75    6
76    6
77    6
78    6
79    6
80    6
81    6
82    6
83    6
84    6
85    6
86    6
87    6
88    6
Name: Facies, dtype: int64